In [ ]:
from model_training.Model import RNNModel
from model_training.Trainer import RNNTrainer
from model_training.Dataset import NavigationDataset
import torch
import pandas as pd
from data_generation.Environment import Environment
from data_generation.Agent import Agent
import matplotlib.pyplot as plt
import pickle

In [ ]:
data = pd.read_csv("data/square_1000traj_50steps.csv")
device = "cuda" if torch.cuda.is_available() else "cpu"
dataset = NavigationDataset(data)
dataloader = dataset.get_dataloader(batch_size=64, shuffle=True)
dataloader_iter = iter(dataloader)
# model = RNNModel(device=device)
with open("models/rnn_sq_model.pkl", "rb") as f:
    model = pickle.load(f)
model.to(device)
trainer = RNNTrainer(model, device=device)
trainer.train(dataloader, n_epochs=1000, verbose=True)
model.to("cpu")
with open("models/rnn_sq_model.pkl", "wb") as f:
    pickle.dump(model, f)

In [ ]:
with open("models/rnn_sq_model.pkl", "rb") as f:
    model = pickle.load(f)
print(len(model.loss_history[0]))

In [ ]:

env_type = 'triangle'

training_data = pd.read_csv(f"data/{env_type}_1000traj_50steps.csv")
print(training_data[(training_data['step'] > 30) & (training_data['run_id'] > 50) & (training_data['collision'] == True) ].iloc[0])
training_data = training_data[training_data['run_id'] == 54]
training_data['run_id'] = 0
training_data = NavigationDataset(training_data)
with open(f"models/rnn_{env_type[0:2]}_model.pkl", "rb") as f:
    model = pickle.load(f)


training_data_loader = training_data.get_dataloader(batch_size=1, shuffle=True)

model.to("cpu")
model.device = "cpu"
# predict the sample data
model.eval()
outputs = []
activations = []
with torch.no_grad():
    for batch in training_data_loader:
        X, Y, lengths, run_ids = batch
        X = X[0]
        Y = Y[0]
        lengths = lengths[0]
        activations, outputs = model(X, lengths)
        break

print(activations.shape)
# agent.visualize_path()
# visualize the true path and predicted path

env = Environment(3 if env_type == 'triangle' else 4, 1, 1)
x, y = env.env.exterior.xy

pred_path = outputs.numpy()
true_path = Y.numpy()
plt.figure(figsize=(6, 6))
plt.fill(x, y, alpha=0.5, ec='black', label='Environment')
plt.scatter(pred_path[0, 0], pred_path[0, 1], color='green', label='Start')
plt.plot(true_path[:, 0], true_path[:, 1], label='True Path', color='green', linestyle='--')
plt.scatter(true_path[1:, 0], true_path[1:, 1], color='green', s=15)
plt.plot(pred_path[:, 0], pred_path[:, 1], label='Predicted Path', color='pink')
plt.scatter(pred_path[1:, 0], pred_path[1:, 1], color='red', s=15)
plt.scatter(pred_path[-1, 0], pred_path[-1, 1], color='red')
plt.scatter(true_path[-1, 0], true_path[-1, 1], color='red')
plt.title('Predicted Path by RNN Model')
plt.xlabel('X Position')
plt.ylabel('Y Position')
plt.legend()
plt.savefig(f'tex/figures/rnn_{env_type}_model_prediction_good.png')

            